### SOC Stock Model Work

The plan: 
- **Gather the covariates** using the SCORPAN model 
- **add harmonized and transformed soc** (response)
- **clean and process training data** -- Take out variables with moderate to high amnts of missing data, impute variables w low to mod
- **Handle categorical variables** --> Convert text/categorical variables (e.g., moist_color_name, friability) to numeric codes or one-hot encoding.
- **Handle numeric predictors**. Scale/normalize if using regression methods sensitive to magnitude (optional, regression kriging is usually OK without scaling).Ensure no extreme outliers that could bias the model.

In [1]:
## Prepare Training Data to estimate SOC stock throughout Angola
import pandas as pd

training_data = pd.read_csv("/Users/inesschwartz/Desktop/training_data_table_final.csv")

In [2]:
training_data.columns

Index(['site_info_id', 'X_coord', 'Y_coord', 'profile', 'district', 'MRVBF',
       'RLD', 'aspect', 'aspect_classes', 'aspect_cos', 'aspect_sin',
       'dem_filledfiltered', 'flow_accumulation', 'relief', 'ridge_levels',
       'roughness', 'slope', 'twi_300m', 'valleydepth2', 'aspect_label',
       'annual_mean_temp', 'annual_precip2', 'isothermality_32733',
       'max_temp_warmest_month32733', 'mean_temp_driest_quarter32733',
       'mean_temp_warmest_quarter32733', 'mean_temp_wettest_quarter32733',
       'min_temp_coldest_month32733', 'precip_coldest_quarter32733',
       'precip_driest_month32733', 'precip_driest_quarter32733',
       'precip_seasonality2', 'precip_warmest_quarter32733',
       'precip_wettest_month32733', 'precip_wettest_quarter32733',
       'temp_annual_range32733', 'temp_seasonality32733', 'landsurface_value',
       'landsurface_label', 'eco_value', 'eco_subclass_clean',
       'eco_class_clean', 'eco_division_clean', 'eco_subclass_code',
       'eco_class

identify numeric vs categorical variables

In [6]:
training_data.columns

Index(['site_info_id', 'X_coord', 'Y_coord', 'profile', 'district', 'MRVBF',
       'RLD', 'aspect', 'aspect_classes', 'aspect_cos', 'aspect_sin',
       'dem_filledfiltered', 'flow_accumulation', 'relief', 'ridge_levels',
       'roughness', 'slope', 'twi_300m', 'valleydepth2', 'annual_mean_temp',
       'annual_precip2', 'isothermality_32733', 'max_temp_warmest_month32733',
       'mean_temp_driest_quarter32733', 'mean_temp_warmest_quarter32733',
       'mean_temp_wettest_quarter32733', 'min_temp_coldest_month32733',
       'precip_coldest_quarter32733', 'precip_driest_month32733',
       'precip_driest_quarter32733', 'precip_seasonality2',
       'precip_warmest_quarter32733', 'precip_wettest_month32733',
       'precip_wettest_quarter32733', 'temp_annual_range32733',
       'temp_seasonality32733', 'landsurface_value', 'landsurface_label',
       'eco_value', 'eco_subclass_clean', 'eco_class_clean',
       'eco_division_clean', 'eco_subclass_code', 'eco_class_code',
       'eco_div

In [12]:
## identify numerica vs categorical variables

import pandas as pd

# Load your dataset
training_data

# Separate numeric and categorical predictors
numeric_cols = training_data.select_dtypes(include=['float64', 'int64']).columns.tolist()
categorical_cols = training_data.select_dtypes(include=['object']).columns.tolist()

# Remove response variable from predictors
numeric_cols.remove('log_soc_stock')  # response variable

print("Numeric predictors:")
print(numeric_cols)

print("\nCategorical predictors:")
print(categorical_cols)


Numeric predictors:
['MRVBF', 'RLD', 'aspect', 'aspect_classes', 'aspect_cos', 'aspect_sin', 'dem_filledfiltered', 'flow_accumulation', 'relief', 'ridge_levels', 'roughness', 'slope', 'twi_300m', 'valleydepth2', 'annual_mean_temp', 'annual_precip2', 'isothermality_32733', 'max_temp_warmest_month32733', 'mean_temp_driest_quarter32733', 'mean_temp_warmest_quarter32733', 'mean_temp_wettest_quarter32733', 'min_temp_coldest_month32733', 'precip_coldest_quarter32733', 'precip_driest_month32733', 'precip_driest_quarter32733', 'precip_seasonality2', 'precip_warmest_quarter32733', 'precip_wettest_month32733', 'precip_wettest_quarter32733', 'temp_annual_range32733', 'temp_seasonality32733', 'landsurface_value', 'eco_value', 'eco_subclass_code', 'eco_class_code', 'eco_division_code', 'faosoil_id']

Categorical predictors:
['DOMSOI', 'africa_lithology_90m.img.vat_lithology']


In [11]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# ======================================================
# --- Handle missing data and drop unwanted columns ---
# ======================================================

# Drop specific unwanted columns
cols_to_drop = ['profile', 'site_info_id', 'district', 'landsurface_label', 'eco_subclass_clean', 'eco_class_clean', 'eco_division_clean', 'FAOSOIL', 'X_coord', 'Y_coord']
training_data1 = training_data
training_data1.drop(columns=cols_to_drop, inplace=True, errors='ignore')


# ======================================================
# --- Feature selection with categorical + numeric ---
# ======================================================

# Separate target
y = training_data1['log_soc_stock']
X = training_data1.drop(columns=['log_soc_stock'], errors='ignore')

# Automatically detect categorical and numeric columns
all_categorical = X.select_dtypes(include=['object', 'category']).columns.tolist()
numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()

# Safeguard: keep only categorical columns with <= 50 unique categories
cat_limit = 50
categorical_cols = [col for col in all_categorical if X[col].nunique() <= cat_limit]

print("Categorical columns included:", categorical_cols)
print("Categorical columns skipped (too many categories):",
      [col for col in all_categorical if col not in categorical_cols])
print("Numeric columns:", numeric_cols)

# Preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
        ('num', 'passthrough', numeric_cols)
    ],
    remainder='drop'
)

# Model
model = RandomForestRegressor(random_state=42)

# Pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

# Fit model
pipeline.fit(X, y)

# Get feature names after OHE
ohe_feature_names = []
if categorical_cols:
    ohe_feature_names = pipeline.named_steps['preprocessor'] \
        .named_transformers_['cat'] \
        .get_feature_names_out(categorical_cols)

all_feature_names = np.concatenate([ohe_feature_names, numeric_cols])

# Feature importances
importances = pipeline.named_steps['model'].feature_importances_
feature_importance = pd.DataFrame({
    'feature': all_feature_names,
    'importance': importances
}).sort_values(by='importance', ascending=False)

print("\nTop 25 Features by Importance:")
print(feature_importance.head(25))


Categorical columns included: ['DOMSOI', 'africa_lithology_90m.img.vat_lithology']
Categorical columns skipped (too many categories): []
Numeric columns: ['MRVBF', 'RLD', 'aspect', 'aspect_classes', 'aspect_cos', 'aspect_sin', 'dem_filledfiltered', 'flow_accumulation', 'relief', 'ridge_levels', 'roughness', 'slope', 'twi_300m', 'valleydepth2', 'annual_mean_temp', 'annual_precip2', 'isothermality_32733', 'max_temp_warmest_month32733', 'mean_temp_driest_quarter32733', 'mean_temp_warmest_quarter32733', 'mean_temp_wettest_quarter32733', 'min_temp_coldest_month32733', 'precip_coldest_quarter32733', 'precip_driest_month32733', 'precip_driest_quarter32733', 'precip_seasonality2', 'precip_warmest_quarter32733', 'precip_wettest_month32733', 'precip_wettest_quarter32733', 'temp_annual_range32733', 'temp_seasonality32733', 'landsurface_value', 'eco_value', 'eco_subclass_code', 'eco_class_code', 'eco_division_code', 'faosoil_id']

Top 25 Features by Importance:
                           feature  

## creating prediction grid

What script does:
- 1 km hex grid — matches fine-resolution DEM, soil, ecosystems layers. 
    - Explicit raster type lists for continuous 1 km, continuous bioclimatic (~18.5 km), categorical 1 km, categorical lithology (~50 km)
- Raster resolution check (xres, yres) — distinguishes whether raster is finer/equal or coarser than hex.
- Continuous layers:
    - Hex smaller than raster → value at centroid.
    - Hex equal/finer → zonal mean.

- Categorical layers:
    - Hex smaller than raster → value at centroid.
    - Hex equal/finer → majority value.
- Chunked processing — memory-efficient.
- Saves outputs in pickle, geopackage, CSV.


In [1]:
import os
import geopandas as gpd
import rioxarray as rxr
import xarray as xr
from rasterio.enums import Resampling

# ------------------------------
# 1) Paths & parameters
# ------------------------------

# Angola boundary (projected in EPSG:32733)
angola_boundary_path = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/angola_boundaries_32733.gpkg"

# Covariate folders
terrain_folder = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/dem_1km"
bioclim_folder = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/bioclimatic32733/" #18.5km

categorical_rasters = {
    "labelled_ecosystems32733": "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/labelled_ecosystems32733_1km.tif",
    "landsurfaceforms": "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/landsurfaceforms/landsurfaceforms_1km.tif",
    "lithology_raster": "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/lithology_rasterized.tif", #50km
    "soil_raster": "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/angola_soil_data_raster_1km.tif", #band = faosoil 
}

In [7]:
import rasterio, numpy as np
with rasterio.open("/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/lithology_rasterized.tif") as src:
    arr = src.read(1, masked=True)
    print(np.unique(arr.compressed()))


[ 1.  3.  4.  5.  6.  7. 12. 14. 16. 20.]


In [9]:
# ------------------------------
# 2) Load Angola boundary
# ------------------------------
angola_gdf = gpd.read_file(angola_boundary_path).to_crs(epsg=32733)

# ------------------------------
# 3) List rasters
# ------------------------------
def list_tifs(folder):
    return sorted([os.path.join(folder, f) for f in os.listdir(folder)
                   if f.lower().endswith(".tif") and not f.startswith("._")])

terrain_rasters = list_tifs(terrain_folder)     # 1 km continuous
bioclim_rasters = list_tifs(bioclim_folder)     # ~18.5 km continuous

all_covariates = terrain_rasters + bioclim_rasters + list(categorical_rasters.values())


In [10]:
# ------------------------------
# 4) Resampling map & types
# ------------------------------
resampling_map = {
    "nearest": Resampling.nearest,
    "bilinear": Resampling.bilinear,
    "cubic": Resampling.cubic
}

categorical_paths = list(categorical_rasters.values())
continuous_paths = [p for p in all_covariates if p not in categorical_paths]

# ------------------------------
# 5) Template raster (DEM 1 km)
# ------------------------------
template = rxr.open_rasterio(terrain_rasters[0], masked=True)


In [11]:
# ------------------------------
# 6) Harmonise & stack
# ------------------------------
stack_list = []

for path in all_covariates:
    name = os.path.splitext(os.path.basename(path))[0]
    da = rxr.open_rasterio(path, masked=True)

    # Choose resampling method
    if path in categorical_paths:
        resample_method = resampling_map["nearest"]
    else:
        resample_method = resampling_map["bilinear"]

    # Reproject/resample to match template
    da = da.rio.reproject_match(template, resampling=resample_method) #bioclim = bilinear resampling → 1 km grid, lithology = nearest-neighbour resampling → 1 km grid.

    # Clip to Angola
    da = da.rio.clip(angola_gdf.geometry, angola_gdf.crs)

    # Squeeze single-band rasters and rename
    da = da.squeeze(drop=True).rename(name)

    stack_list.append(da)

# Combine into a single xarray Dataset (each covariate = one band)
covariate_stack = xr.merge(stack_list)

In [12]:
# ------------------------------
# 7) Optional: write multiband GeoTIFF
# ------------------------------
covariate_stack.to_array().rio.to_raster("angola_covariate_stack.tif", compress="lzw")

print("✅ Covariate raster stack saved as angola_covariate_stack.tif")
print("Variables:", list(covariate_stack.data_vars))

# ------------------------------
# 8) Flatten stack to 2D array for prediction (optional)
# ------------------------------
# Each row = pixel, each column = covariate
flat_stack = covariate_stack.to_array().stack(pixel=("y", "x")).transpose("pixel", "variable")
flat_stack_df = flat_stack.to_pandas()
print("✅ Flattened covariate stack ready for prediction")

✅ Covariate raster stack saved as angola_covariate_stack.tif
Variables: ['MRVBF_1km', 'RLD_1km', 'aspect_1km', 'aspect_cos_1km', 'aspect_sin_1km', 'dem_filledfiltered_1km', 'flow_accumulation_1km', 'relief_1km', 'ridge_levels_1km', 'roughness_1km', 'slope_1km', 'twi_300m_1km', 'valleydepth2_1km', 'annual_mean_temp', 'annual_precip2', 'isothermality_32733', 'max_temp_warmest_month32733', 'mean_temp_driest_quarter32733', 'mean_temp_warmest_quarter32733', 'mean_temp_wettest_quarter32733', 'min_temp_coldest_month32733', 'precip_coldest_quarter32733', 'precip_driest_month32733', 'precip_driest_quarter32733', 'precip_seasonality2', 'precip_warmest_quarter32733', 'precip_wettest_month32733', 'precip_wettest_quarter32733', 'temp_annual_range32733', 'temp_seasonality32733', 'labelled_ecosystems32733_1km', 'landsurfaceforms_1km', 'lithology_rasterized', 'angola_soil_data_raster_1km']
✅ Flattened covariate stack ready for prediction


In [14]:
## COMPARING TRAINING DATA AND RASTER PREDICTION GRID

import pandas as pd

# ------------------------------
# 1) Training data
# ------------------------------
training_data = pd.read_csv("/Users/inesschwartz/Desktop/training_data.csv")
training_columns = list(training_data.columns)
print("Training data columns:")
print(training_columns)
print(f"Total columns: {len(training_columns)}\n")

# ------------------------------
# 2) Covariate stack
# ------------------------------
# Assuming you already have covariate_stack loaded with xarray
# If not, uncomment and load:
# import rioxarray as rxr
# import xarray as xr
# covariate_stack = rxr.open_rasterio("angola_covariate_stack.tif", masked=True)

stack_columns = list(covariate_stack.data_vars)
print("Prediction grid (covariate stack) variables:")
print(stack_columns)
print(f"Total variables: {len(stack_columns)}\n")

# ------------------------------
# 3) Compare
# ------------------------------
missing_in_stack = [c for c in training_columns if c not in stack_columns]
missing_in_training = [c for c in stack_columns if c not in training_columns]

print("Columns in training data but missing in prediction grid:", missing_in_stack)
print("Columns in prediction grid but missing in training data:", missing_in_training)


Training data columns:
['site_info_id', 'X_coord', 'Y_coord', 'profile', 'district', 'MRVBF', 'RLD', 'aspect', 'aspect_cos', 'aspect_sin', 'dem', 'flow_accumulation', 'relief', 'ridge_levels', 'roughness', 'slope', 'twi', 'valleydepth', 'annual_mean_temp', 'annual_precip', 'isothermality', 'max_temp_warmest_month', 'mean_temp_driest_quarter', 'mean_temp_warmest_quarter', 'mean_temp_wettest_quarter', 'min_temp_coldest_month', 'precip_coldest_quarter', 'precip_driest_month', 'precip_driest_quarter', 'precip_seasonality', 'precip_warmest_quarter', 'precip_wettest_month', 'precip_wettest_quarter', 'temp_annual_range', 'temp_seasonality', 'landsurface_value', 'landsurface_label', 'eco_value', 'eco_subclass_code', 'eco_class_code', 'eco_division_code', 'FAOSOIL', 'DOMSOI', 'faosoil_id', 'lithology', 'log_soc_stock']
Total columns: 46

Prediction grid (covariate stack) variables:
['MRVBF_1km', 'RLD_1km', 'aspect_1km', 'aspect_cos_1km', 'aspect_sin_1km', 'dem_filledfiltered_1km', 'flow_accumul

PREVIOUS WORKFLOW (HEX GRID NOT RASTER GRID)

In [6]:
#previous workflow hex grid

# ------------------------------------------------------------------
# 1) Paths & parameters
# ------------------------------------------------------------------
angola_boundary_path = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/angola_boundaries_32733.gpkg"

terrain_folder = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/DEM_characteristics_300m"
bioclim_folder = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/bioclimatic32733/"

categorical_rasters = {
    "labelled_ecosystems32733": "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/labelled_ecosystems32733_1km.tif",
    "landsurfaceforms":          "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/landsurfaceforms/landsurfaceforms_1km.tif",
    "lithology_raster":          "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/lithology_rasterized.tif",
    "soil_raster":               "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/angola_soil_data_raster_1km.tif",
}

def list_tifs(folder):
    return sorted([
        os.path.join(folder, f)
        for f in os.listdir(folder)
        if f.lower().endswith(".tif") and not f.startswith("._")
    ])

terrain_rasters = list_tifs(terrain_folder)   # 1 km continuous
bioclim_rasters = list_tifs(bioclim_folder)   # ~18.5 km continuous
covariate_paths = terrain_rasters + bioclim_rasters + list(categorical_rasters.values())

# Hex grid parameters
hex_size = 1000      # 1 km
chunk_size = 2000    # hexes per processing chunk
# # ------------------------------------------------------------------
# 2) Load Angola boundary vector
# ------------------------------------------------------------------
angola_gdf = gpd.read_file(angola_boundary_path)
angola_gdf = angola_gdf.to_crs(epsg=32733)  # ensure projected in meters
angola_boundary = angola_gdf.unary_union     # single polygon for intersections
angola_crs = angola_gdf.crs

# ------------------------------------------------------------------
# 3) Build 1 km hexagon grid
# ------------------------------------------------------------------
xmin, ymin, xmax, ymax = angola_boundary.bounds
dx = np.sqrt(3) * hex_size
dy = 1.5 * hex_size
hexagons = []
x = xmin
row = 0

while x < xmax + dx:
    y = ymin - hex_size if row % 2 else ymin
    while y < ymax + dy:
        hexagon = Polygon([
            (x + hex_size * np.cos(a), y + hex_size * np.sin(a))
            for a in np.linspace(0, 2*np.pi, 7)[:-1]
        ])
        # Only keep hexes that intersect Angola
        if hexagon.intersects(angola_boundary):
            hexagons.append(hexagon.intersection(angola_boundary))
        y += dy
    x += dx
    row += 1

grid_gdf = gpd.GeoDataFrame(geometry=hexagons, crs=angola_crs)
grid_gdf["hex_id"] = range(1, len(grid_gdf) + 1)

# Optional: log column to flag coarse layers
grid_gdf["coarse_flag"] = ""

print(f"✅ Hex grid created with {len(grid_gdf)} hexagons")
#  ------------------------------------------------------------------
# 4) Chunked zonal statistics
# ------------------------------------------------------------------
# Prepare columns
for rpath in covariate_paths:
    name = [k for k, v in categorical_rasters.items() if v == rpath]
    col_name = name[0] if name else os.path.splitext(os.path.basename(rpath))[0]
    grid_gdf[col_name] = np.nan

n_chunks = int(np.ceil(len(grid_gdf) / chunk_size))
print(f"Processing {len(grid_gdf)} hexagons in {n_chunks} chunks...")

for i, chunk_idx in enumerate(np.array_split(grid_gdf.index, n_chunks), start=1):
    print(f"  Chunk {i}/{n_chunks}")
    subset = grid_gdf.loc[chunk_idx]

    for rpath in covariate_paths:
        name = [k for k, v in categorical_rasters.items() if v == rpath]
        col_name = name[0] if name else os.path.splitext(os.path.basename(rpath))[0]

        with rasterio.open(rpath) as src:
            nodata = src.nodata

        # ----------------------
        # Continuous rasters
        # ----------------------
        if rpath in continuous_1km:
            # 1 km or finer -> zonal mean
            stats = zonal_stats(subset, rpath, stats="mean", nodata=nodata)
            grid_gdf.loc[chunk_idx, col_name] = [s["mean"] if s else np.nan for s in stats]

        elif rpath in continuous_bioclim:
            # Coarser than hex -> take raster value at centroid
            values = []
            with rasterio.open(rpath) as src:
                for geom in subset.geometry:
                    centroid = geom.centroid
                    row, col = src.index(centroid.x, centroid.y)
                    val = src.read(1)[row, col]
                    values.append(float(val) if val != nodata else np.nan)
            grid_gdf.loc[chunk_idx, col_name] = values
            grid_gdf.loc[chunk_idx, "coarse_flag"] += f"{col_name} "

        # ----------------------
        # Categorical rasters
        # ----------------------
        elif rpath in categorical_1km:
            stats = zonal_stats(subset, rpath, categorical=True, nodata=nodata)
            grid_gdf.loc[chunk_idx, col_name] = [
                max(s, key=s.get) if s else np.nan for s in stats
            ]

        elif rpath in categorical_lithology:
            values = []
            with rasterio.open(rpath) as src:
                for geom in subset.geometry:
                    centroid = geom.centroid
                    row, col = src.index(centroid.x, centroid.y)
                    val = src.read(1)[row, col]
                    values.append(int(val) if val != nodata else np.nan)
            grid_gdf.loc[chunk_idx, col_name] = values
            grid_gdf.loc[chunk_idx, "coarse_flag"] += f"{col_name} "

# ------------------------------------------------------------------
# 5) Save outputs
# ------------------------------------------------------------------
grid_gdf.to_pickle("angola_SOC_prediction_grid.pkl")
grid_gdf.to_file("angola_SOC_prediction_grid.gpkg", driver="GPKG")
grid_gdf.to_csv("angola_SOC_prediction_grid.csv", index=False)

print("✅ Saved outputs: .pkl, .gpkg, .csv")
print("✅ Coarse raster layers per hex recorded in 'coarse_flag' column")


KeyboardInterrupt: 

## ANNK to build SOC map

In [ ]:
# annk_workflow
import os
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from pykrige.ok import OrdinaryKriging
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
import joblib

# -------------------
# USER PATHS / SETTINGS
# -------------------
TRAINING_CSV = "/Users/inesschwartz/Desktop/training_data_table_final.csv"
PREDICTION_GRID_GPKG = "angola_SOC_prediction_grid.gpkg"  # adjust if path differs
PREDICTION_GRID_LAYER = None  # None will use default/first layer
OUTPUT_PREFIX = "angola_ANNK"
SOC_COL = "log_soc_stock"            # change if different
X_COL = "X_coord"          # change if different
Y_COL = "Y_coord"          # change if different
RANDOM_SEED = 42
TEST_SIZE = 1/3            # validation fraction
# ANN hyperparameters (tune if desired)
ANN_PARAMS = {
    "hidden_layer_sizes": (100, 50),  # example; tune for your problem
    "activation": "relu",
    "solver": "adam",
    "alpha": 1e-4,
    "learning_rate": "adaptive",
    "max_iter": 300,
    "early_stopping": True,
    "random_state": RANDOM_SEED,
    "verbose": True
}
# pykrige settings
VARIOGRAM_MODEL = "spherical"  # options: 'linear','power','gaussian','spherical','exponential'
N_SEARCH = 10  # pykrige internal parameter (keeps run time reasonable)

In [ ]:
# -------------------
# 1) Load training data
# -------------------
df = pd.read_csv(TRAINING_CSV)
required_cols = {SOC_COL, X_COL, Y_COL}
if not required_cols.issubset(df.columns):
    missing = required_cols - set(df.columns)
    raise ValueError(f"Missing required columns in training CSV: {missing}")

# identify covariate columns (exclude coords and SOC and any identifier-like columns)
exclude = {SOC_COL, X_COL, Y_COL}
# also exclude common ID columns if present
for c in ["profile", "district", "site_info_id"]:
    exclude.add(c)
covariate_cols = [c for c in df.columns if c not in exclude]

if len(covariate_cols) == 0:
    raise ValueError("No covariate columns detected in training CSV. Update covariate detection or file.")

print(f"Detected {len(covariate_cols)} covariates: {covariate_cols[:10]}{'...' if len(covariate_cols)>10 else ''}")

# Drop rows with no coordinates or no SOC
df = df.dropna(subset=[SOC_COL, X_COL, Y_COL]).copy()

# -------------------
# 2) Train/validation split (2/3 train, 1/3 validation)
# -------------------
train_df, val_df = train_test_split(df, test_size=TEST_SIZE, random_state=RANDOM_SEED)
print(f"Training samples: {len(train_df)}, Validation samples: {len(val_df)}")


In [ ]:
# -------------------
# 3) Preprocessing pipeline
#    - auto-detect numeric vs categorical covariates from training data
# -------------------
# choose numeric vs categorical by pandas dtypes (object/category -> categorical)
numeric_cols = [c for c in covariate_cols if pd.api.types.is_numeric_dtype(df[c])]
categorical_cols = [c for c in covariate_cols if c not in numeric_cols]

print(f"Numeric covariates: {numeric_cols[:10]}{'...' if len(numeric_cols)>10 else ''}")
print(f"Categorical covariates: {categorical_cols[:10]}{'...' if len(categorical_cols)>10 else ''}")

numeric_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])

categorical_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse=False))
])

preprocessor = ColumnTransformer(transformers=[
    ("num", numeric_pipeline, numeric_cols),
    ("cat", categorical_pipeline, categorical_cols)
], remainder="drop")

# Fit preprocessor on training data
X_train_raw = train_df[covariate_cols]
y_train = train_df[SOC_COL].values

X_val_raw = val_df[covariate_cols]
y_val = val_df[SOC_COL].values

preprocessor.fit(X_train_raw)

X_train = preprocessor.transform(X_train_raw)
X_val = preprocessor.transform(X_val_raw)

In [ ]:
# -------------------
# 4) Train MLPRegressor (ANN)
# -------------------
ann = MLPRegressor(**ANN_PARAMS)
print("Training ANN...")
ann.fit(X_train, y_train)

# ANN predictions on train and validation
yhat_train = ann.predict(X_train)
yhat_val = ann.predict(X_val)

def print_metrics(y_true, y_pred, label):
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2 = r2_score(y_true, y_pred)
    print(f"{label} RMSE: {rmse:.4f} | R2: {r2:.4f}")
    return rmse, r2

print_metrics(y_train, yhat_train, "Train (ANN)")
print_metrics(y_val, yhat_val, "Validation (ANN)")

In [ ]:
# -------------------
# 5) Compute residuals at training locations (observed - ANN_pred)
# -------------------
# Need ANN predictions at all training points (use the same preprocessor)
X_train_all = preprocessor.transform(df[covariate_cols])
df["ann_pred"] = ann.predict(X_train_all)
df["residual"] = df[SOC_COL] - df["ann_pred"]

# Subset for kriging: use training points only (the ANN was trained on train_df, but kriging uses all available observations)
# Option: you could use only train_df residuals or all residuals; here I will use all residuals where covariates available
krige_df = df.dropna(subset=["residual", X_COL, Y_COL]).copy()
kx = krige_df[X_COL].values
ky = krige_df[Y_COL].values
kz = krige_df["residual"].values
print(f"Fitting kriging to {len(kz)} residual points")

# -------------------
# 6) Fit Ordinary Kriging on residuals (pykrige)
# -------------------
# pykrige expects 1d arrays
# For speed, estimate variogram/model by trying default; pykrige will fit sill/range if possible.
OK = OrdinaryKriging(
    kx, ky, kz,
    variogram_model=VARIOGRAM_MODEL,
    verbose=False,
    enable_plotting=False,
    nlags=10
)

In [ ]:
# -------------------
# 7) Load prediction grid and predict ANN on grid covariates
# -------------------
grid = gpd.read_file(PREDICTION_GRID_GPKG, layer=PREDICTION_GRID_LAYER)
print(f"Loaded prediction grid: {len(grid)} polygons")

# compute centroids for prediction points
grid = grid.to_crs(epsg=dem_crs_to_epsg(grid.crs) if False else grid.crs)  # no-op; keep consistent, helper below removed
grid["centroid"] = grid.geometry.centroid
grid["x_centroid"] = grid.centroid.x
grid["y_centroid"] = grid.centroid.y

# Important: ensure the grid has the same covariate columns as training covariates
missing_covs = [c for c in covariate_cols if c not in grid.columns]
if missing_covs:
    raise ValueError(f"The prediction grid is missing covariate columns required for ANN: {missing_covs}")

# prepare feature matrix for grid and transform
X_grid_raw = grid[covariate_cols]
X_grid = preprocessor.transform(X_grid_raw)
# ANN prediction on grid
grid["ann_pred"] = ann.predict(X_grid)

In [ ]:
# -------------------
# 8) Krige residuals to grid centroids
# -------------------
# pykrige's execute with 'points' returns arrays of shape (n_points,)
xpoints = grid["x_centroid"].values
ypoints = grid["y_centroid"].values

print("Kriging residuals to grid centroids (this may take some time)...")
z_pred, ss = OK.execute("points", xpoints, ypoints)  # z_pred: predicted residuals, ss: variance
grid["kriged_residual"] = np.array(z_pred).ravel()
grid["kriging_var"] = np.array(ss).ravel()


In [ ]:
# -------------------
# 9) Final ANNK prediction (ANN prediction + kriged residual)
# -------------------
grid["soc_annk"] = grid["ann_pred"] + grid["kriged_residual"]

# -------------------
# 10) Save outputs and model objects (pickle)
# -------------------
# Save GeoPackage with predictions
out_gpkg = f"{OUTPUT_PREFIX}_prediction_grid.gpkg"
grid.drop(columns=["centroid"]).to_file(out_gpkg, driver="GPKG")
print(f"Saved GeoPackage: {out_gpkg}")

# Save CSV with attributes (geometry as WKT)
grid_out = grid.copy()
grid_out["geometry_wkt"] = grid_out.geometry.to_wkt()
grid_out.drop(columns="geometry").to_csv(f"{OUTPUT_PREFIX}_prediction_grid.csv", index=False)
print(f"Saved CSV: {OUTPUT_PREFIX}_prediction_grid.csv")

# Save models and preprocessor for reproducibility
joblib.dump(ann, f"{OUTPUT_PREFIX}_ann_model.joblib")
joblib.dump(preprocessor, f"{OUTPUT_PREFIX}_preprocessor.joblib")
joblib.dump(OK, f"{OUTPUT_PREFIX}_kriging_ok.joblib")  # note: pickling pykrige objects may be large but useful
print("Saved ANN model, preprocessor, and kriging object via joblib")

# optional: compute approximate validation after kriging (if we can get residual kriged back to validation points)
# predict ANN for validation points, krige residuals at validation coords and compare final predictions to observed
X_val_all = preprocessor.transform(val_df[covariate_cols])
val_df = val_df.copy()
val_df["ann_only"] = ann.predict(X_val_all)
# krige residuals at val coords
z_val_pred, ss_val = OK.execute("points", val_df[X_COL].values, val_df[Y_COL].values)
val_df["kriged_residual"] = np.array(z_val_pred).ravel()
val_df["annk_pred"] = val_df["ann_only"] + val_df["kriged_residual"]
print("Validation performance (ANN only):")
print_metrics(val_df[SOC_COL].values, val_df["ann_only"].values, "Validation (ANN only)")
print("Validation performance (ANN + OK residuals):")
print_metrics(val_df[SOC_COL].values, val_df["annk_pred"].values, "Validation (ANNK)")

print("Done.")